In [ ]:
import pandas as pd
import numpy as np
import glob
import seaborn as sn
import math
import csv
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torch.optim as optim
import torchvision
from tqdm.notebook import trange, tqdm
from einops import rearrange
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import ShuffleSplit
import copy
from itertools import cycle
import time
import argparse
import os
from sklearn.metrics import accuracy_score
#import tensorboard_logger as tb_logger
import shap
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap

### <font color=brown> **Class 1: xrb-sirf45theta, Class 2: SSP** </font>

## ***Preprocessing full spectral information***

In [ ]:
fileLocation = '/home/local/AD/ssaha3/CIDER/Data/contflux/class1/*.csv'
processed_files_class1 = glob.glob(fileLocation)
cnt = 0
for file in processed_files_class1:
    cnt = cnt + 1
    #print(file)

print(cnt)

In [ ]:
fileLocation = '/home/local/AD/ssaha3/CIDER/Data/contflux/class2/*.csv'
processed_files_class2 = glob.glob(fileLocation)
cnt = 0
for file in processed_files_class2:
    cnt = cnt + 1
    #print(file)

print(cnt)

In [ ]:
def calculate_log_10(inp_dataframe):
    temp = inp_dataframe
    result = np.log10(temp + 1e-01)
    final_dataframe = pd.DataFrame(result, columns = ['incid_luminosity', 'attenuated_incid_luminosity', 'diffuse_luminosity', 'diffuse_cont_luminosity'])
    return final_dataframe

In [ ]:
myList = []
y = []
wavelength_cont = []
for file in processed_files_class1:
  csv_data = pd.read_csv(file, dtype={'wavelength': str})
  wavelength_cont = csv_data['wavelength'].tolist()
  csv_data = csv_data.drop(['wavelength'], axis=1)
  csv_data = calculate_log_10(csv_data)
  all_data_tensor = torch.tensor(csv_data.values.astype(np.float32))
  myList.append(all_data_tensor)
  y.append(0)

for file in processed_files_class2:
  csv_data = pd.read_csv(file)
  csv_data = csv_data.drop(['wavelength'], axis=1)
  csv_data = calculate_log_10(csv_data)
  all_data_tensor = torch.tensor(csv_data.values.astype(np.float32))
  myList.append(all_data_tensor)
  y.append(1)

X1 = torch.stack(myList)
#X2 = rearrange(X2, 'd0 d1 d2 -> d0 (d1 d2)')
Y = torch.tensor(y)

In [ ]:
#print(len(wavelength_cont))
#print(wavelength_cont[2000])

In [ ]:
mean, std = 20.6, 20.6
temp_X1 = torch.empty_like(X1)
print(temp_X1.shape)
for i in range(X1.shape[0]):
    temp_X1[i][:] = (X1[i][:] - mean)/std
temp_X1 = temp_X1.view(294,1,2008,4)
torch.min(temp_X1), torch.max(temp_X1)

In [ ]:
X1 = temp_X1
print(X1.shape, X1.dtype)
print(Y.shape, Y.dtype)

In [ ]:
fileLocation = '/home/local/AD/ssaha3/CIDER/Data/lineflux/class1/*.csv'
processed_files_class1 = glob.glob(fileLocation)
cnt = 0
for file in processed_files_class1:
    cnt = cnt + 1
    #print(file)

print(cnt)

In [ ]:
fileLocation = '/home/local/AD/ssaha3/CIDER/Data/lineflux/class2/*.csv'
processed_files_class2 = glob.glob(fileLocation)
cnt = 0
for file in processed_files_class2:
    cnt = cnt + 1
    #print(file)

print(cnt)

In [ ]:
def calculate_log_10(inp_dataframe):
  temp = inp_dataframe
  result = np.log10(temp + 1e-01)
  final_dataframe = pd.DataFrame(result, columns = ['luminosity'])
  return final_dataframe

In [ ]:
myList = []
y = []
wavelength_line = []
for file in processed_files_class1:
  csv_data = pd.read_csv(file, dtype={'wavelength': str})
  wavelength_line = csv_data['wavelength'].tolist()
  csv_data = csv_data.drop(['wavelength'], axis=1)
  csv_data = calculate_log_10(csv_data)
  all_data_tensor = torch.tensor(csv_data.values.astype(np.float32))
  myList.append(all_data_tensor)
  y.append(0)

for file in processed_files_class2:
  csv_data = pd.read_csv(file)
  csv_data = csv_data.drop(['wavelength'], axis=1)
  csv_data = calculate_log_10(csv_data)
  all_data_tensor = torch.tensor(csv_data.values.astype(np.float32))
  myList.append(all_data_tensor)
  y.append(1)

X2 = torch.stack(myList)
#X2 = rearrange(X2, 'd0 d1 d2 -> d0 (d1 d2)')
Y = torch.tensor(y)

In [ ]:
print(len(wavelength_line))

In [ ]:
#  normalize data to be in -1 to 1 range

mean, std = 19.4, 19.4 
temp_X2 = torch.empty_like(X2)
print(temp_X2.shape)
for i in range(X2.shape[0]):
    temp_X2[i][:] = (X2[i][:] - mean)/std
temp_X2 = temp_X2.view(294,1,143,1)
torch.min(temp_X2), torch.max(temp_X2)

In [ ]:
print(X2.shape, X2.dtype)
print(Y.shape, Y.dtype)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
from torch.utils.data import Dataset, DataLoader
class DataBuilderContflux(Dataset):
    def __init__(self, data, inp_set):
        self.x, self.y = X1[inp_set], Y[inp_set]
        self.y = self.y
        self.len=self.x.shape[0]
    def __getitem__(self,index):
        y = self.y[index].type(torch.LongTensor)
        return self.x[index], y
    def __len__(self):
        return self.len

In [ ]:
from torch.utils.data import Dataset, DataLoader
class DataBuilderLineflux(Dataset):
    def __init__(self, data, inp_set):
        self.x, self.y = X2[inp_set], Y[inp_set]
        self.y = self.y
        self.len=self.x.shape[0]
    def __getitem__(self,index):
        y = self.y[index].type(torch.LongTensor)
        return self.x[index], y
    def __len__(self):
        return self.len

In [ ]:
#'''
kfold = 5
batch_size = 4
ss = ShuffleSplit(n_splits=kfold, test_size=0.20, random_state=1)
i = 1

for train_index, test_index in ss.split(X1, Y):
  temp = train_index
  t = int(X1.shape[0] * 0.60)
  i_t = t
  #print(t, temp.shape[0]-i_t)
  train_set = temp[0:t]
  val_set = temp[i_t:temp.shape[0]]
  test_set = test_index
  #print(len(train_set))
  train_x = DataBuilderContflux(X1, train_set)
  train_loader_contflux = DataLoader(dataset=train_x,batch_size=batch_size, shuffle=False)

  val_x = DataBuilderContflux(X1, val_set)
  val_loader_contflux = DataLoader(dataset=val_x,batch_size=batch_size, shuffle=False)

  test_x = DataBuilderContflux(X1, test_set)
  test_loader_contflux = DataLoader(dataset=test_x,batch_size=batch_size, shuffle=False)
#'''

In [ ]:
#utility function
def check_batch_data(inp_loader):
  for batch_idx, (x,y) in enumerate(inp_loader):
      x = x.to(device)
      y = y.to(device)
      print(x)
      print(x.shape, y.shape)
      print(x.dtype, y.dtype)
      break
  print(batch_idx)
  return x, y
x1,y1 = check_batch_data(test_loader_contflux)

In [ ]:
'''
#utility function
def check_batch_data(inp_loader):
  for batch_idx, (x,y) in enumerate(inp_loader):
      x = x.to(device)
      y = y.to(device)
      print(x.shape, y.shape)
      print(x.dtype, y.dtype)
      break
  print(batch_idx)
  return x, y
x2,y2 = check_batch_data(train_loader_lineflux)
'''

In [ ]:
'''
print(x1.shape, y1.shape)
print(x1.dtype, y1.dtype)
'''

In [ ]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)  # Input channels: 4, Output channels: 16
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)  # Input channels: 16, Output channels: 32
        self.bn2 = nn.BatchNorm2d(32)
        self.fc = nn.Linear(32*2008*4, 143)  # Input size: 32 * 2008, Output size: 143

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        x = x.view(x.size(0), 143)  # Reshape to the desired output shape
        return x




model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.load_state_dict(torch.load("/home/local/AD/ssaha3/CIDER/Code/baseline_01/model/baseline_01/trial_03/fold1/ckpt_199.pt", map_location=torch.device(device)))

In [ ]:
batch = next(iter(test_loader_contflux))

In [ ]:
x, _ = batch
x = x.to(device)
print(x.shape)

In [ ]:
out = model(x)
out.shape

In [ ]:
test_samples = []
for batch_idx, (x,y) in enumerate(test_loader_contflux):
    x = x.to(device)
    test_samples.append(x)
    
all_test_samples = torch.cat(test_samples)

In [ ]:
all_test_samples.shape

In [ ]:
explainer = shap.DeepExplainer(model, x.to(device))
shap_val = explainer.shap_values(all_test_samples)

In [ ]:
shap_val[0].shape

In [ ]:
'''
import pickle
with open('wavelength_line.pkl', 'wb') as file:
    pickle.dump(wavelength_line, file)

with open('wavelength_cont.pkl', 'wb') as file:
    pickle.dump(wavelength_cont, file)
'''

In [ ]:
'''
with open('shap_val.pkl', 'rb') as file:
    shap_data = pickle.load(file)
'''

In [ ]:
shap_data[0].shape

In [ ]:
#np.squeeze(shap_val[0], axis=1).shape

In [ ]:
def get_mean_shap_val():
    regression_out_length = 143
    all_mean_shap_val = []
    for i in range(0,regression_out_length):
        all_mean_shap_val.append(np.mean(np.squeeze(shap_val[i], axis=1), axis=0))

    # Print the shape of the resulting tensor
    #print(all_mean_shap_val[0].shape)
    #print(all_mean_shap_val[0][:,0].shape)
    return all_mean_shap_val

In [ ]:
'''
t1 = get_mean_shap_val()
t1[0].shape
len(t1)
'''

In [ ]:
'''
std_feat_imp_score = {}
for i in range(0, batch_sz):
    for j in range(0,2008):
        if j not in std_feat_imp_score.keys():
            std_feat_imp_score[j] = 0
        else:
            std_feat_imp_score[j] += np.std(shap_val[0][i][0][j])
'''

In [ ]:
'''
batch_sz = 4
regression_out_length = 143
for ind in range(0, regression_out_length):
    mean_feat_imp_score = {}
    for i in range(0, batch_sz):
        for j in range(0,2008):
            if j not in mean_feat_imp_score.keys():
                mean_feat_imp_score[j] = 0
            else:
                mean_feat_imp_score[j] += np.mean(shap_val[ind][i][0][j])
    sorted_dict = dict(sorted(mean_feat_imp_score.items(), key=lambda x: x[1], reverse=True))
    if ind not in total_mean_feat_imp_score.keys():
        total_mean_feat_imp_score[ind] = 0
    else:
        total_mean_feat_imp_score[ind] = sorted_dict
'''

In [ ]:
'''
batch_sz = 4
regression_out_length = 143

total_mean_feat_imp_score = np.zeros((regression_out_length, 2008))

for ind in range(regression_out_length):
    mean_feat_imp_score = {}
    for i in range(0,batch_sz):
        for j in range(2008):
            if j not in mean_feat_imp_score.keys():
                mean_feat_imp_score[j] = 0
            else:
                mean_feat_imp_score[j] += np.mean(shap_val[ind][i][0][j])
    
    sorted_dict = dict(sorted(mean_feat_imp_score.items(), key=lambda x: x[1], reverse=True))
    
    total_mean_feat_imp_score[ind] = list(mean_feat_imp_score.values())

#print(total_mean_feat_imp_score)
'''

In [ ]:
'''
for key in sorted_mean_feat_imp_score:
    value1 = sorted_mean_feat_imp_score[key]
    value2 = std_feat_imp_score[key]
    print(key, value1, value2)
'''

In [ ]:
def get_pos_log():
    np.seterr(divide = 'ignore')
    array_shape = (143, 2008, 4)
    all_shap_val = np.empty(array_shape)
    all_mean_shap_val = get_mean_shap_val()
    #print(all_mean_shap_val[0].shape)
    
    for i, element in enumerate(all_mean_shap_val):
        all_shap_val[i] = element

    #abs_val_log = np.log(np.abs(all_shap_val))
    #positive_values = all_shap_val[all_shap_val > 0]
    positive_vals = np.where(all_shap_val > 0.0, all_shap_val, 0.0)
    positive_val_log = np.where(positive_vals > 0.0, np.log(positive_vals), positive_vals)
    return positive_val_log

In [ ]:
def get_neg_log():
    np.seterr(divide = 'ignore')
    array_shape = (143, 2008, 4)
    all_shap_val = np.empty(array_shape)
    all_mean_shap_val = get_mean_shap_val()
    
    for i, element in enumerate(all_mean_shap_val):
        all_shap_val[i] = element

    negative_vals = np.where(all_shap_val < 0.0, all_shap_val, 0.0)
    negative_val_log = np.where(negative_vals < 0.0, np.log(np.abs(negative_vals)), negative_vals)  #np.log(1) = 0, so applying np.log to negative values will not cause any 0 
    return negative_val_log

In [ ]:
'''
if np.log(9.11671790e-04) > np.log(8.54650344e-04):
    print('y')
if np.log(8.54650344e-04) > np.log(1.66684640e-04):
    print('y')
    
a = np.abs(-2.44357070e-06)
b = np.abs(-8.08775597e-06)
print(np.log(a), np.log(b))
if -2.44357070e-06 > -8.08775597e-06:
    print('y')
if np.log(a) > np.log(b):
    print('*y')
'''

In [ ]:
'''
arr = np.array([5, -2, 10, -6, 3, 0, -8, 7, -1, 4])

# Get the indices of the three smallest values
smallest_indices = np.argsort(arr)[:3]

# Get the other indices
other_indices = np.arange(len(arr))
other_indices = np.delete(other_indices, smallest_indices)

# Place zero at the other_indices
arr[other_indices] = 0

print("Updated array:", arr)
'''

In [ ]:
'''
feat_ind = 1
n = 10
negative_vals_log = get_neg_log()
temp = negative_vals_log[:,:,feat_ind]
for i in range(143):
    sorted_arr = np.sort(temp[i])[::-1]
    print(sorted_arr)
    # Select the three largest negative values while ignoring zero
    negative_values = sorted_arr[sorted_arr < 0.0]
    # Get the top n indices
    largest_negative_values = negative_values[:n]
    print(largest_negative_values)
    # Get the indices of the selected values in the original array
    selected_indices = np.where(temp[i] == largest_negative_values[:, None])[1]
    print(selected_indices)
    other_indices = np.arange(len(temp[i]))
    other_indices = np.delete(other_indices, selected_indices)
    temp[i][other_indices] = 0.0
'''

In [ ]:
'''
arr = np.array([-5, -2, -10, -6, -3, 0, -8, -7, -4, -9])

# Sort the array in descending order
sorted_arr = np.sort(arr)[::-1]

# Select the three largest negative values while ignoring zero
negative_values = sorted_arr[sorted_arr < 0]
largest_negative_values = negative_values[:3]

# Get the indices of the selected values in the original array
selected_indices = np.where(arr == largest_negative_values[:, None])[1]
other_indices = np.arange(len(arr))
other_indices = np.delete(other_indices, selected_indices)

# Place zero at the other_indices
arr[other_indices] = 0


print("Selected Indices:", selected_indices)
print("Updated array:", arr)
'''

In [ ]:
'''
for i in range(0,143):
    cnt = 0
    ind = []
    for j in range(0,2008):
        if temp[i][j] != 0.0:
            cnt += 1
            ind.append(j)
    print(i, cnt, ind)
'''

In [ ]:
'''
min_val = -np.inf
if 0.0 > min_val:
    print('y')
'''

# Processing Positive Feature Importance Score

In [ ]:
def get_top_n_feat_score(feat_ind, total_top_n):
    #feat_ind = 0
    top_n = 1               # to select top 1 value for each lineflux wavelength
    positive_val_log = get_pos_log()
    temp = positive_val_log[:,:,feat_ind]

    #selecting top 1 value for each lineflux wavelength
    for i in range(143):
        sorted_arr = np.sort(temp[i])[::-1]
        # Get the top 10 indices
        top_n_indices = np.argsort(temp[i])[:top_n]
        #print(top_n_indices) 

        other_indices = np.arange(len(temp[i]))
        other_indices = np.delete(other_indices, top_n_indices)
        temp[i][other_indices] = 0.0



    list_of_top_1 = []
    for i in range(143):
        list_of_top_1.append([x for x in temp[i] if x < 0.0][0])

    #selecting top 10 values from all 143 lineflux wavelengths
    #total_top_n = 100         # to select total top 10 values from [143,2008] matrix
    list_of_top_n_indices = np.argsort(list_of_top_1)[:total_top_n]
    #print(list_of_top_n_indices)
    for i in range(143):
        if i not in list_of_top_n_indices:
            temp[i] = 0
        
        else:  #setting values other than zero to -200.0 so that they can be easily identified
            for index, value in enumerate(temp[i]):
                if value != 0.0:
                    print(value, wavelength_line[i], wavelength_cont[index])
                    #temp[i][index] = 200
                    #print(temp[i])
    #print(temp)
    return temp
    #print(cnt)

In [ ]:
feat_score = get_top_n_feat_score(0, 10)

In [ ]:
'''
total_top_n_indices = []
for i in range(143):
    sorted_arr = np.sort(feat_score[i])[::-1]
    # Get the top 10 indices
    top_n_indices = np.argsort(feat_score[i])[:1]
    total_top_n_indices.append(top_n_indices[0])
print(total_top_n_indices)
'''

In [ ]:
'''
total_top_n_indices = []
for i in range(143):
    sorted_arr = np.sort(temp[i])[::-1]
    # Get the top 10 indices
    top_n_indices = np.argsort(temp[i])[:top_n]
    total_top_n_indices.append(top_n_indices[0])
print(total_top_n_indices)
'''

In [ ]:
'''
for i in range(len(total_top_n_indices)):
    if total_top_n_indices[i] > 0:
        temp[i][total_top_n_indices[i]] = - 200.0
'''

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_stat(feat_ind, tot_top_n):
    
    feat_score = get_top_n_feat_score(feat_ind, tot_top_n)
    fig, ax1 = plt.subplots(figsize=(20, 8))
    
    # Create a colormap for non-zero values
    cmap = plt.get_cmap('viridis')  # Change 'viridis' to the desired colormap for non-zero values
    
    # Create scatter points based on feat_score
    y_coords, x_coords = np.indices(feat_score.shape)
    
    # Set zero values to white
    feat_score_with_white = np.where(feat_score == 0, 1.0, feat_score)  # Map zero values to 1.0 (white)
    
    # Use feat_score_with_white as the color intensity
    scatter = ax1.scatter(x_coords, y_coords, c=feat_score_with_white.flatten(), cmap=cmap, marker='s', s=list(feat_score.flatten()*(-100.0)))
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    
    # Set tick parameters
    ax1.tick_params(axis='both', which='major', labelsize=35, length=0)
    plt.xlabel('Spectral Lines (Contflux)', fontsize=35, fontweight='bold')
    plt.ylabel('Spectral Lines (Lineflux)', fontsize=35, fontweight='bold')

    # Set title based on feat_ind
    titles = [
        'incid luminosity (positive values)',
        'attenuated_incid luminosity (positive values)',
        'diffuse luminosity (positive values)',
        'diffuse continuum luminosity (positive values)'
    ]
    plt.title(titles[feat_ind], fontsize=35, fontweight='bold')

    # Adjust layout and display the scatterplot
    plt.tight_layout()
    plt.show()


In [ ]:
_ = widgets.interact(
    plot_stat, 
    feat_ind = list(np.arange(0,4)),
    tot_top_n = list(np.arange(0,100))
)

# Processing Negative Feature Importance Score

In [ ]:
def get_top_n_neg_feat_score(feat_ind, total_top_n):
    #feat_ind = 0
    top_n = 1               # to select top 1 value for each lineflux wavelength
    negative_vals_log = get_neg_log()
    temp = negative_vals_log[:,:,feat_ind]

    #selecting top 1 value for each lineflux wavelength
    for i in range(143):
        sorted_arr = np.sort(temp[i])[::-1]
        #print(sorted_arr)
        # Select the three largest negative values while ignoring zero
        negative_values = sorted_arr[sorted_arr < 0.0]
        # Get the top n indices
        largest_negative_values = negative_values[:top_n]
        #print(largest_negative_values)
        # Get the indices of the selected values in the original array
        selected_indices = np.where(temp[i] == largest_negative_values[:, None])[1]
        #print(selected_indices)
        other_indices = np.arange(len(temp[i]))
        other_indices = np.delete(other_indices, selected_indices)
        temp[i][other_indices] = 0.0



    list_of_top_1 = []
    for i in range(143):
        list_of_top_1.append([x for x in temp[i] if x < 0.0][0])

    #selecting top 10 values from all 143 lineflux wavelengths
    #total_top_n = 100         # to select total top 10 values from [143,2008] matrix
    list_of_top_n_indices = np.argsort(list_of_top_1)[:total_top_n]
    #print(list_of_top_n_indices)
    for i in range(143):
        if i not in list_of_top_n_indices:
            temp[i] = 0
        
        else:  #setting values other than zero to -200.0 so that they can be easily identified
            for index, value in enumerate(temp[i]):
                if value != 0.0:
                    print(value, wavelength_line[i], wavelength_cont[index])
                    #temp[i][index] = 200
                    #print(temp[i])

    #print(temp)
    return temp
    #print(cnt)

In [ ]:
def plot_stat_neg(feat_ind, tot_top_n):
    
    feat_score = get_top_n_neg_feat_score(feat_ind, tot_top_n)
    fig, ax1 = plt.subplots(figsize=(20, 8))
    
    # Create a colormap for non-zero values
    cmap = plt.get_cmap('viridis')  # Change 'viridis' to the desired colormap for non-zero values
    
    # Create scatter points based on feat_score
    y_coords, x_coords = np.indices(feat_score.shape)
    
    # Set zero values to white
    feat_score_with_white = np.where(feat_score == 0, 1.0, feat_score)  # Map zero values to 1.0 (white)
    
    # Use feat_score_with_white as the color intensity
    scatter = ax1.scatter(x_coords, y_coords, c=feat_score_with_white.flatten(), cmap=cmap, marker='s', s=list(feat_score.flatten()*(-100.0)))
    
    # Add colorbar
    cbar = plt.colorbar(scatter)
    
    # Set tick parameters
    ax1.tick_params(axis='both', which='major', labelsize=35, length=0)
    plt.xlabel('Spectral Lines (Contflux)', fontsize=35, fontweight='bold')
    plt.ylabel('Spectral Lines (Lineflux)', fontsize=35, fontweight='bold')

    # Set title based on feat_ind
    titles = [
        'incid luminosity (negative values)',
        'attenuated_incid luminosity (negative values)',
        'diffuse luminosity (negative values)',
        'diffuse continuum luminosity (negative values)'
    ]
    plt.title(titles[feat_ind], fontsize=35, fontweight='bold')

    # Adjust layout and display the scatterplot
    plt.tight_layout()
    plt.show()


In [ ]:
_ = widgets.interact(
    plot_stat_neg, 
    feat_ind = list(np.arange(0,4)),
    tot_top_n = list(np.arange(0,100))
)